In [1]:
from shared import corpus, tokenizers, datasets

text = corpus.shakespeare()
tokenizer = tokenizers.unique_chars(text)
train, test = datasets.causal(tokenizer.encode(text))

/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset tiny_shakespeare (/Users/cztomsik/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)
100%|██████████| 3/3 [00:00<00:00, 1282.01it/s]


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GPT(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, num_layers=4):
        super().__init__()
        self.transformer = nn.Sequential(
            nn.Embedding(vocab_size, embed_dim),
            nn.Sequential(*[Layer(embed_dim) for _ in range(num_layers)]),
            nn.LayerNorm(embed_dim),
        )
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, x):
        return self.lm_head(self.transformer(x))

class Layer(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.q = nn.Linear(embed_dim, embed_dim, bias=False)
        self.v = nn.Linear(embed_dim, embed_dim, bias=False)
        self.proj = nn.Linear(embed_dim, embed_dim, bias=False)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.GELU(approximate="tanh"),
            nn.Linear(4 * embed_dim, embed_dim),
        )
        self.proj = nn.Linear(embed_dim, embed_dim, bias=False)

    def forward(self, x):
        B, T, C = x.shape
        xn = self.ln1(x)
        q = self.q(xn)
        v = self.v(F.pad(xn, (0, 0, 1, -1)))
        attn = self.proj(torch.sigmoid(q) * v)

        x = x + attn
        x = x + self.mlp(self.ln2(x))
        return x


import lightning as pl
import torch.utils.data as data
device = "mps"
batch_size = 36

class Model(pl.LightningModule):
    def __init__(self, vocab_size, lr=0.007):
        super().__init__()
        self.lr = lr
        self.model = GPT(vocab_size)
    
    def forward(self, x, y=None):
        logits = self.model(x)
        return logits if y is None else F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1), ignore_index=-1)

    def train_dataloader(self):
        return data.DataLoader(train, batch_size=batch_size, num_workers=0, sampler=data.RandomSampler(train, False, 6_000))

    def training_step(self, batch, batch_idx):
        return self(*batch)

    def val_dataloader(self):
        return data.DataLoader(test, batch_size=batch_size, num_workers=0)
    
    def validation_step(self, batch, batch_idx):
        loss = self(*batch)
        self.log("test_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_epoch_end(self, outs):
        with torch.no_grad():
            print(self.generate("And now", 64))

    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=self.lr)
        sched = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.95, last_epoch=-1)
        return [optim], [sched]

    # inspired by https://github.com/karpathy/minGPT/blob/master/mingpt/model.py
    @torch.no_grad()
    def generate(self, str, max_new_tokens, top_k=10):
        ids = torch.tensor(tokenizer.encode(str), dtype=torch.long).unsqueeze(0).to(self.device)
        for _ in range(max_new_tokens):
            #out = self(ids[:, -block_size:])
            out = self(ids)
            logits = out[:, -1, :]
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float("Inf")
            step_res = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
            # auto-regression
            ids = torch.cat((ids, step_res), dim=1)
        return tokenizer.decode(ids[0].tolist())

pl.seed_everything(89026614)
model = Model(tokenizer.vocab_size)
trainer = pl.Trainer(max_epochs=25, enable_progress_bar=True, accelerator="gpu" if device == "cuda" else device)
trainer.fit(model)

Global seed set to 89026614
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(

  | Name  | Type | Params
-------------------------------
0 | model | GPT  | 190 K 
-------------------------------
190 K     Trainable params
0         Non-trainable params
190 K     Total params
0.764     Total estimated model p

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 29.56it/s]And now:cncvL?pZElbQ3pjoxbgvX-?
vgC:GRzjyWgUBO3OchYRy,F d3tKI,F qsQhKZV
                                                                           

/Users/cztomsik/miniconda3/envs/torch-mps/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 209/209 [00:05<00:00, 35.85it/s, loss=1.67, v_num=43]And now, what here's for it would stop are and with the paice my fair,

Epoch 1: 100%|██████████| 209/209 [00:05<00:00, 37.97it/s, loss=1.53, v_num=43, test_loss=1.830]And now, to disdang althen
And by him.

GLOUCESTER:
Go things, it is su
Epoch 2: 100%|██████████| 209/209 [00:06<00:00, 30.04it/s, loss=1.46, v_num=43, test_loss=1.680]And now his part actions the mounty!'
This set the grounds
Will hence,

Epoch 4: 100%|██████████| 209/209 [00:09<00:00, 20.97it/s, loss=1.41, v_num=43, test_loss=1.530]And now?

MENENIUS:
I will be gry brother,
And shall spurpose.

GLOUCES
Epoch 5: 100%|██████████| 209/209 [00:11<00:00, 18.71it/s, loss=1.39, v_num=43, test_loss=1.540]And now, ho! what, methought in mind of my son
Is but the sorne at a ch
Epoch 6: 100%|██████████| 209/209 [00:11<00:00, 18.98it/s, loss=1.38, v_num=43, test_loss=1.510]And now?

DUKE VINCENTIO:
Then I
morning the that your fintal approfane
Epoch 7: 

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 209/209 [00:10<00:00, 20.74it/s, loss=1.29, v_num=43, test_loss=1.420]


In [3]:
print(model.generate("O God, O God!", 650))

O God, O God! what wise, inconspirator:
And thy warrant. Somerset a pleased, only till you have not frowns are them the period to do nor hands upon, this so the sanst me to his heart will, and him, my lord's say 'Ay.
I authority:
If I means over business?
And woodfellow? where's my honour, much,
Have your garpends the searchs disgraces may should sleep out of your gentlemen,--

QUEEN:
All the writ, a month the whose my face;
Saddle will chide, thus. He do you better sleep himself in such a dream'd,
I have, I would done;
We must some presence as at his sudden so beseech you?

RIVERS:
How so still hold, man?
Once the fires to the daughters o' the blood
Hast
